# Information Retrieval Evaluation Pipeline
This notebook provides a template for evaluating query reformulation techniques using PyTerrier.
Pipeline stages: Dataset Loading → Preprocessing → Query Reformulation → Retrieval → Evaluation



## 1. Dataset Loading
This can be switched with another dataset, possibly requiring conversion to this format.


In [1]:
import pandas as pd
import pyterrier as pt
from pathlib import Path
from datasets import load_dataset

class DatasetComponents:
    """Container for dataset components that must be provided"""
    def __init__(self, corpus_iter, queries_df, qrels_df):
        self.corpus_iter = corpus_iter  # Iterator yielding {'docno': str, 'text': str}
        self.queries_df = queries_df    # DataFrame with columns ['qid', 'query']
        self.qrels_df = qrels_df        # DataFrame with columns ['qid', 'docno', 'label']

def load_pt_dataset():
    """Load codec dataset"""
    docs = load_dataset("macavaney/codec")["train"]
    qrels = load_dataset('irds/codec', 'qrels')
    queries = load_dataset('irds/codec', 'queries')

    # Convert dataset to correct format
    corpus_iter = ({'docno': str(doc['id']), 'text': doc['contents']} for doc in docs)

    queries_df = pd.DataFrame(queries)[['query_id', 'query']]
    queries_df.columns = ['qid', 'query']

    qrels_df = pd.DataFrame(qrels)[['query_id', 'doc_id', 'relevance']]
    qrels_df.columns = ['qid', 'docno', 'label']

    return DatasetComponents(corpus_iter, queries_df, qrels_df)

# Load the dataset
data = load_pt_dataset()

C:\Users\thein\anaconda3\envs\irllm\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 2. Preprocessing Pipeline
Currently, this does no preprocessing.

In [3]:
if not pt.java.started():
    pt.java.init()

tokeniser = pt.java.autoclass("org.terrier.indexing.tokenisation.Tokeniser").getTokeniser()
def strip_markup(text):
    return " ".join(tokeniser.getTokens(text))


def preprocess_text(text: str) -> str:
    """Placeholder for text preprocessing logic"""
    return text

def preprocess_corpus(corpus_iter):
    """Generator that applies preprocessing to each document"""
    for doc in corpus_iter:
        yield {
            'docno': doc['docno'],
            'text': preprocess_text(doc['text'])
        }

def preprocess_queries(queries_df):
    """Apply preprocessing to queries dataframe"""
    queries_df = queries_df.copy()
    queries_df['query'] = queries_df['query'].apply(strip_markup)
    return queries_df

# Apply preprocessing while maintaining iterator
preprocessed_corpus = preprocess_corpus(data.corpus_iter)
preprocessed_queries = preprocess_queries(data.queries_df)

Java started and loaded: pyterrier.java, pyterrier.terrier.java [version=5.11 (build: craig.macdonald 2025-01-13 21:29), helper_version=0.0.8]


## 3. Query Reformulation

In [4]:
def reformulate_queries(queries_df):
    """Placeholder for query reformulation techniques"""
    modified_queries = queries_df.copy()
    # Placeholder for actual reformulation
    return modified_queries

reformulated_queries = reformulate_queries(preprocessed_queries)

## 4. Indexing Pipeline

In [5]:
index_path = Path.cwd() / "index"
index_ref = None

# Check if valid index exists
if (index_path / "data.properties").exists():
    try:
        index_ref = pt.IndexFactory.of(str(index_path))
        print(f"Loaded existing index from {index_path}")

        # Verify index contains documents
        if index_ref.getCollectionStatistics().getNumberOfDocuments() == 0:
            raise ValueError("Empty index - will rebuild")

    except Exception as e:
        print(f"Index loading failed ({str(e)}), rebuilding...")
        import shutil
        shutil.rmtree(index_path)
        index_ref = None

# Build new index if needed
if index_ref is None:
    print("Building new index...")
    index_ref = pt.index.IterDictIndexer(
        str(index_path),
        meta={"docno": 32, "text": 131072},
        type=pt.index.IndexingType.CLASSIC
    ).index(preprocessed_corpus)
    print(f"Built new index at {index_path}")

    print(index_ref.getCollectionStatistics())


11:55:00.694 [main] WARN org.terrier.structures.BaseCompressingMetaIndex -- Structure meta reading data file directly from disk (SLOW) - try index.meta.data-source=fileinmem in the index properties file. 1.4 GiB of memory would be required.
Loaded existing index from C:\Users\thein\OneDrive\Documents\InformationRetrieval\llm-query-rewriting\index


Number of documents: 729824
Number of terms: 941881
Number of postings: 159765825
Number of fields: 0
Number of tokens: 273318564
Field names: []
Positions:   false



In [1]:
# Function to delete the index in case it should be recreated
import shutil
from pathlib import Path

def delete_index(index_path):
    """Deletes the index at the specified path."""
    if index_path.exists():
        shutil.rmtree(index_path)
        print(f"Deleted index at {index_path}")
    else:
        print("No index found to delete.")

Deleted index at C:\Users\thein\OneDrive\Documents\InformationRetrieval\llm-query-rewriting\index


## 5. Retrieval Setup
Currently, using BM25 for retrieval.

In [10]:
bm25 = pt.BatchRetrieve(
    index_ref,
    wmodel="BM25",
    metadata=["docno", "text"],
    properties={"termpipelines": ""},
    controls={"qe": "off"}
)

C:\Users\thein\AppData\Local\Temp\ipykernel_23224\2758154324.py:1: DeprecationWarning: Call to deprecated class BatchRetrieve. (use pt.terrier.Retriever() instead) -- Deprecated since version 0.11.0.
  bm25 = pt.BatchRetrieve(


## 6. Evaluation Pipeline

In [ ]:
# Run evaluation experiment
# metrics: https://pyterrier.readthedocs.io/en/latest/experiments.html#available-evaluation-measures
import os
import json
from IPython.core.display import display, HTML


# Directory containing JSON files
results_dir = "./results"

# Evaluation metrics
eval_metrics = ["map", "ndcg_cut_10", "P_10", "recall_100", "recip_rank"]

results = pt.Experiment(
    [bm25],
    preprocessed_queries,
    data.qrels_df,
    eval_metrics,
    names=["BM25 Baseline"],
    baseline=0
)

# Display the results
print("Results for no query reformulation:")
display(HTML(results.to_html(index=False)))


# Loop through JSON files in the directory
for filename in os.listdir(results_dir):
    if filename.endswith(".json"):
        file_path = os.path.join(results_dir, filename)
        try:
            with open(file_path, "r", encoding="utf-8") as f:
                data_json = json.load(f)

            # Convert JSON data to DataFrame
            queries_df = pd.DataFrame(data_json)[["query_id", "query"]]
            queries_df.columns = ["qid", "query"]
            queries_df['query'] = queries_df['query'].apply(strip_markup)

            print(f"Processing file: {filename}")

            results = pt.Experiment(
                [bm25],
                queries_df,
                data.qrels_df,
                eval_metrics,
                names=["BM25 Baseline"],
                baseline=0
            )

            # Display the results
            display(HTML(results.to_html(index=False)))

        except Exception as e:
            print(f"Error processing {filename}: {e}")


C:\Users\thein\AppData\Local\Temp\ipykernel_23224\1737640262.py:5: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython.display
  from IPython.core.display import display, HTML
